In [1]:
## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hii'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-grep

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..
## You will need to run jupyter from that directory.
## source $TEMP_BIN/_dir-aliases.bash
source _dir-aliases.bash
alias | wc -l

8


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 7071)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-7071

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-grep/test-7071


In [6]:
# Count aliases proper
alias | wc -l

9


In [7]:
# Count functions
typeset -f | egrep '^\w+' | wc -l

2


In [8]:
# check for a modern version of grep. For example,
#
# $ grep -V
# grep (GNU grep) 2.4.2
#
# Copyright 1988, 1992-1999, 2000 Free Software Foundation, Inc.
# ...
#
# In contrast, here's an old verion (e.g., under medusa):
#
# $ grep -V
# GNU grep version 2.0
# usage: grep [-[[AB] ]<num>] [-[CEFGVchilnqsvwx]] [-[ef]] <expr> [<files...>]

skip_dirs=""
if [[ $(grep --version) =~ Copyright.*2[0-9][0-9][0-9] ]]; then skip_dirs="-d skip"; fi
grep -V

grep (GNU grep) 3.7
Copyright (C) 2021 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.

Written by Mike Haertel and others; see
<https://git.sv.gnu.org/cgit/grep.git/tree/AUTHORS>.


In [9]:
# Grep settings
# TODO: use gr and gr_ throughout for consistency
# TODO: use -P flag (i.e.,  --perl-regexp) w/ grep rather than egrep
# Notes:
# - MY_GREP_OPTIONS used instead of GREP_OPTIONS since grep interprets latter
#   -n       show line numbers
#   -d skip  skip directories (i.e., don't treat as files)
#   -s       suppress error messages (e.g., unreadable files)
#   -E       extended regex support (i.e., old egrep)
# - /bin/grep used to avoid alias and to allow for use with exec
# - egrep is normally used unless the pattern will never use extended regex's
## TODO: quiet-unalias grep
## TODO: add alias for resolving grep binary with fallback to "command grep"
GREP="/bin/grep"
EGREP="$GREP -E"
export MY_GREP_OPTIONS="-n $skip_dirs -s"
# effing shellcheck (SC2086: Double quote to prevent globbing)
# shellcheck disable=SC2086

In [10]:
{
function gr () { $GREP $MY_GREP_OPTIONS -i "$@"; }
function gr- () { $GREP $MY_GREP_OPTIONS "$@"; }
SORT_COL2="--key=2"
# grep-unique(pattern, file, ...): count occurrence of pattern in file...
function grep-unique () { $GREP -c $MY_GREP_OPTIONS "$@" | $GREP -v ":0" | sort -rn $SORT_COL2 -t':'; }
# grep-missing(pattern, file, ...): show files without pattern 
# TODO: archive
function grep-missing () { $GREP -c $MY_GREP_OPTIONS "$@" | $GREP ":0"; }
alias gu='grep-unique -i'
## OLD
## alias gru='gu'
alias gu-='grep-unique'
# gu-all: run gu over all files in current dir
# TODO: archive
function gu-all () { grep-unique "$@" ./* | $PAGER; }
#
function gu- () { $GREP -c $MY_GREP_OPTIONS "$@" | $GREP -v ":0"; }
#
# grepl(pattern, [other_grep_args]): invokes grep over PATTERN and OTHER_GREP_ARGS and then pipes into less for PATTERN
# NOTE: actually uses egrep
# TODO: use more general way to ensure pattern given last while readily extractable for less -p usage
## OLD: function grep-to-less () { $GREP $MY_GREP_OPTIONS "$@" | $PAGER -p"$1"; }
function grep-to-less () { $EGREP $MY_GREP_OPTIONS "$@" | $PAGER_NOEXIT -p"$1"; }
alias grepl-='grep-to-less'
function grepl () { pattern="$1"; shift; grep-to-less "$pattern" -i "$@"; }
}

In [11]:
## SETTING UP FOR GREP TEST
rm -rf ./*

# THE WORD TO BE TESTED - sensitive
mkdir testdir1 testdir2
echo "As grep commands are case sensitive, one of the most useful operators for grep searches as they are sensitive is -i. Instead of printing lowercase results only, the terminal displays both uppercase and lowercase results. The output includes lines with mixed case entries.
" > testgrep1
echo "sensitive sensitive sensitive Sensitive SENSITIVE lorem ipsum" > testgrep2
echo "no mentions here" > testgrep3
echo "Passwords are generally case sensitive" > testgrep4


In [12]:
## TESTING GREP ALIASES

# 0) grep 
grep sensitive testgrep1 testgrep2 testgrep3 testgrep4
linebr
# A) gu (grep-unique i-)
gu sensitive testgrep1 testgrep2 testgrep3 testgrep4
linebr
# B) gu- (grep-unique)
gu- sensitive testgrep1 testgrep2 testgrep3 testgrep4
linebr
# # C) grepl- (grep-to-less)
# grepl- sensitive testgrep1 testgrep2 testgrep3 testgrep4
# ERROR: bash: -psensitive: command not found
#        /bin/grep: write error: Broken pipe

testgrep1:As grep commands are case sensitive, one of the most useful operators for grep searches as they are sensitive is -i. Instead of printing lowercase results only, the terminal displays both uppercase and lowercase results. The output includes lines with mixed case entries.
testgrep2:sensitive sensitive sensitive Sensitive SENSITIVE lorem ipsum
testgrep4:Passwords are generally case sensitive
--------------------------------------------------------------------------------
testgrep4:1
testgrep2:1
testgrep1:1
--------------------------------------------------------------------------------
testgrep4:1
testgrep2:1
testgrep1:1
--------------------------------------------------------------------------------


In [13]:
# TODO: create function for creating gr-xyz aliases
# TODO: -or- create gr-xyz template
## function gr-xyz () { gr- "$@" *.xyz; }

# show-line-context(file, line-num): show 5 lines before LINE-NUM in FILE
# TODO: archive
function show-line-context() { cat -n "$1" | $GREP -B5 "^\W+$2\W"; }

# Helper function for grep-based aliases pipe into less
function gr-less () { gr "$@" | $PAGER; }

In [14]:
# Other grep-related stuff
#
# EX: echo $'1 one\n2 \x80\n3 three' | gr-nonascii => $'2: 2 \x80'
alias grep-nonascii='perlgrep.perl "[\x80-\xFF]"'

In [15]:
## grep-nonascii sensitive - HALTED

In [16]:
# Searching for files
# TODO:
# - specify find options in an environment variable
# - rework in terms of Perl regex? (or use -iregex in place of -iname)
#
function findspec () { if [ "$2" = "" ]; then echo "Usage: findspec dir glob-pattern find-option ... "; else /usr/bin/find "$1" -iname \*"$2"\* "$3" "$4" "$5" "$6" "$7" "$8" "$9" 2>&1 | $GREP -v '^find: '; fi; }
# findspec[-all](dir, pattern, option): find files in directory tried, optionally following links (-all)
function findspec-all () { /usr/bin/find "$1" -follow -iname \*"$2"\* "$3" "$4" "$5" "$6" "$7" "$8" "$9" -print 2>&1 | $GREP -v '^find: '; }
function fs () { findspec . "$@" | $EGREP -iv '(/(backup|build)/)'; } 
function fs-ls () { fs "$@" -exec ls -l {} \; ; }
alias fs-='findspec-all .'
function fs-ext () { find . -iname \*."$1" | $EGREP -iv '(/(backup|build)/)'; } 
# TODO: extend fs-ext to allow for basename pattern (e.g., fs-ext java ImportXML)
function fs-ls- () { fs- "$@" -exec ls -l {} \; ; }
#
findgrep_opts="-in"
#
# NOTE: findgrep macros use $findgrep_opts dynamically (eg, user can change $findgrep_opts)
function findgrep-verbose () { find "$1" -iname \*"$2"\* -print -exec $GREP $findgrep_opts "$3" "$4" "$5" "$6" "$7" "$8" "$9" \{\} \;; }
# findgrep(dir, filename_pattern, line_pattern): $GREP through files in DIR matching FILENAME_PATTERN for LINE_PATTERN
function findgrep () { find "$1" -iname \*"$2"\* -exec $GREP $findgrep_opts "$3" "$4" "$5" "$6" "$7" "$8" "$9" \{\} /dev/null \;; }
function findgrep- () { find "$1" -iname "$2" -print -exec $GREP $findgrep_opts "$3" "$4" "$5" "$6" "$7" "$8" "$9" \{\} \;; }
function findgrep-ext () { local dir="$1"; local ext="$2"; shift; shift; find "$dir" -iname "*.$ext" -exec $GREP $findgrep_opts "$@" \{\}  /dev/null \;; }
# fgr(filename_pattern, line_pattern): $GREP through files matching FILENAME_PATTERN for LINE_PATTERN
function fgr () { findgrep . "$@" | $EGREP -v '((/backup)|(/build))'; }
function fgr-ext () { findgrep-ext . "$@" | $EGREP -v '(/(backup)|(build)/)'; }
alias fgr-py='fgr-ext py'

In [17]:
# fgr-py - HALTED

In [18]:
function prepare-find-files-here () {
    if [ "$1" != "" ]; then
        echo "Error: No arguments accepted; did you mean find-files-here?"
        return
    fi
    local brief_opts="R"
    local full_opts="alR"
    local brief_file="ls-$brief_opts.list"
    local full_file="ls-$full_opts.list"
    local current_files=($full_file $full_file.log $brief_file $brief_file.log)
    # Rename existing files with file date as suffix (TODO move into ./backup)
    # shellcheck disable=SC2068
    rename-with-file-date ${current_files[@]}

    # Perform the actual listings, putting errors in the .log file for each listing
    # Note: If root directory, filters out special directories (TODO: make optional and/or overridable).
    ## TODO: use approach based on filter variable and to avoid redundant hard-coding
    ## TODO: ** resolve intermittent problem when running under /
    if [ "$PWD" = "/" ]; then
        ($NICE $LS -$brief_opts | $EGREP -v '^\.(/(cdrom|dev|media|mnt|proc|run|sys|snap)$)' | perl -pe 's/^\./\n$&/;' > "$brief_file") 2> "$brief_file".log
        ($NICE $LS -$full_opts | $EGREP -v '^\.(/(cdrom|dev|media|mnt|proc|run|sys|snap)$)' | perl -pe 's/^\./\n$&/;' > "$full_file") 2> "$full_file".log
    else
        ($NICE $LS -$brief_opts | perl -pe 's/^\./\n$&/;' > "$brief_file") 2> "$brief_file".log
        ($NICE $LS -$full_opts | perl -pe 's/^\./\n$&/;' > "$full_file") 2> "$full_file".log
    fi;
    
    # shellcheck disable=SC2068
    $LS -lh ${current_files[@]}
}

In [19]:
# TODO: have variant of prepare-find-files that adds .mmmYY suffix to backup
#
# find-files-there(pattern, ls-alR-files): check for PATTERN in LS-ALR-FILES,
# showing the directory, which has trailing ':' in listing (i.e., DIR1:\nentry1\n...entryN\n\nDIR2:\n...)
# exs: "./archive:", "-rwxrwx--- 1 root vboxsf   19125 Dec  7  2013 morph.txt", "-rwxrwx--- 1 root vboxsf      35 Jan 17 12:19 python-notes.txt"
# Note: Perl paragraph-mode search first matches files along with the containing
# subdirectory, and then line-mode search filters out non-matching files.
#
function find-files-there () { perlgrep.perl -para -i "$@" | $EGREP -i '((:$)|('"$1"'))' | $PAGER_NOEXIT -p "$1"; }
function find-files-here () { find-files-there "$1" "$PWD/ls-alR.list"; }
# following variants for sake of tab completion
alias find-files='find-files-here'
alias find-files-='find-files-there'
# TODO: function find-files-dated () { perlgrep.perl -para -i "$@" | $EGREP -i '((:$)|('$1'))' | $PAGER_NOEXIT -p "$1"; }
#
# TODO: add --quiet option to dobackup.sh (and port to bash)
# TODO: function conditional-backup() { if [ -e backup/"$1" ]; then dobackup.sh "$1"; fi; }
alias make-file-listing='listing="ls-aR.list"; dobackup.sh "$listing"; $LS -aR >| "$listing" 2>&1'
